# Aim of this notebook

Proof of concept for a collaborative filtering based recommendation system using Numpy. I plan to implement the model on my own here, and use gradient descent for learning the objective function.

## Non-Weighted Matrix Factorization

For a first try, I will try to implement a non-weighted matrix factorization <br>

3 users, 3 items, 2 features <br>
<br>
Define the user item matrix as A with values in {-1, 0, 1} such that <br>
 1 -> user likes the item <br>
 0 -> user hasn't reviewed the item <br>
-1 -> user doesn't like the item <br>

In [1]:
import numpy as np

#taking the example from https://towardsdatascience.com/recommender-systems-in-python-from-scratch-643c8fc4f704
#
# |U1F1 U1F2 |    |F1I1 F1I2 F1I3 |     |U1I1 U1I2 U1I3|
# |U2F1 U2F2 | *  |F2I1 F2I2 F2I3 |  =  |U2I1 U2I2 U2I3|
# |U3F1 U3F3 |                          |U3I1 U3I2 U3I3|
#

A = np.array([(1,0,0),(-1,0,1), (-1,1,1)])
user_count = A.shape[0]
item_count = A.shape[1]
number_of_features = 3
np.random.seed(1)
user_features = np.random.uniform(low = 0.1, high = 0.9, size = (user_count, number_of_features))
item_features = np.random.uniform(low = 0.1, high = 0.9, size = (item_count,number_of_features))

In [ ]:
#for testing/debugging purposes only
#np.sum((A - np.matmul(user_features, np.transpose(item_features)))**2)
-2 * np.matmul(A - np.matmul(user_features, np.transpose(item_features)), item_features)
-2 * np.matmul(A - np.matmul(user_features, np.transpose(item_features)), user_features)

In [2]:
#function to calculate mean square error between actual user-item matrix and calculated user-item matrix
def MSE(A, user_features, item_features):
    A_calc = np.matmul(user_features, np.transpose(item_features))
    return np.sum((A - A_calc)**2)
    

In [3]:
def calculate_user_feature_gradient(A, user_features, item_features):
    return -2 * np.matmul(A - np.matmul(user_features, np.transpose(item_features)), item_features)

def calculate_item_feature_gradient(A, user_features, item_features):
    return -2 * np.matmul(A - np.matmul(user_features, np.transpose(item_features)), user_features)

def update_user_feature(A, user_features, item_features, learning_rate):
    user_features -= (learning_rate/(2*user_features.shape[0]))*calculate_user_feature_gradient(A, user_features, item_features)
    return user_features

def update_item_feature(A, user_features, item_features, learning_rate):
    item_features -= (learning_rate/(2*item_features.shape[0]))*calculate_item_feature_gradient(A, user_features, item_features)
    return item_features

In [4]:
#let's try training the recommender here
learning_rate = 0.1
for i in range(10000):
    user_features = update_user_feature(A,user_features, item_features, learning_rate)
    item_features = update_item_feature(A, user_features, item_features, learning_rate)
    if(i % 500 == 0):
        print(MSE(A, user_features, item_features))

6.178807264988235
2.1468553096704612
1.6446910411571558
0.41986587119276336
0.05833948514560738
0.25785079552615514
0.014753412872279666
0.00014590743643052
2.3504947055669466e-06
3.480464017392854e-08
4.881401216038528e-10
6.6826423631111585e-12
9.05451292522539e-14
1.2214553421251648e-15
1.6446799716744486e-17
2.212852987001358e-19
2.9754407462597456e-21
4.009909884770188e-23
5.423899504842053e-25
1.1097967155957342e-26


In [5]:
np.matmul(user_features, np.transpose(item_features))

array([[ 1.00000000e+00, -1.11022302e-14,  1.23234756e-14],
       [-1.00000000e+00,  2.70894418e-14,  1.00000000e+00],
       [-1.00000000e+00,  1.00000000e+00,  1.00000000e+00]])